In [1]:
import numpy as np

from dftpy.constants import ENERGY_CONV
from edftpy import io
from edftpy.functional import LocalPP, KEDF, Hartree, XC
from edftpy.optimizer import Optimization
from edftpy.evaluator import EmbedEvaluator, TotalEvaluator
from edftpy.subsystem.subcell import SubCell, GlobalCell
from edftpy.interface import init_graphtopo
from edftpy.mpi import MP, sprint
from edftpy.engine.driver import DriverKS
from edftpy.engine.engine_qe import EngineQE
from edftpy.utils.common import Field, Functional, AbsFunctional

In [2]:
def get_optimizer(cellfile, subkeys, indices):
    #-----------------------------------------------------------------------
    pplist = {'H' : './H_ONCV_PBE-1.2.upf', 'O' : './O_ONCV_PBE-1.2.upf'}
    ecut = 1200*ENERGY_CONV["eV"]["Hartree"]
    cellsplit = [0.5, 0.5, 0.5]
    #-----------------------------------------------------------------------
    ions = io.ase_read(cellfile)
    graphtopo = get_graphtopo([1,]*len(subkeys), parallel = True)
    gsystem = get_gsystem(ions, graphtopo, pplist, ecut)
    drivers = []
    for i, keysys in enumerate(subkeys):
        if graphtopo.isub != i and graphtopo.is_mpi:
            driver = None
        else :
            index = indices[i]
            driver = get_driver(keysys, ions, gsystem.grid, pplist, index, cellsplit, graphtopo)
        drivers.append(driver)

    graphtopo.build_region(grid=gsystem.grid, drivers=drivers)
    opt = Optimization(drivers = drivers, gsystem = gsystem, options={'econv': 1E-6*ions.nat})
    return opt

In [3]:
def get_gsystem(ions, graphtopo, pplist, ecut):
    mp_global = MP(comm = graphtopo.comm, parallel = graphtopo.is_mpi, decomposition = graphtopo.decomposition)
    gsystem = GlobalCell(ions, ecut = ecut, mp = mp_global, graphtopo = graphtopo)
    total_evaluator = get_total_evaluator(ions, gsystem.grid, pplist)
    gsystem.total_evaluator = total_evaluator
    return gsystem

In [4]:
def get_graphtopo(nprocs, parallel = False):
    graphtopo = init_graphtopo(parallel)
    graphtopo.distribute_procs(nprocs)
    return graphtopo

In [5]:
def get_total_evaluator(ions, grid, pplist):
    xc_kwargs = {'xc' : 'PBE'}
    ke_kwargs = {'kedf' : 'GGA', 'k_str' : 'revAPBEK'}
    pseudo = LocalPP(grid = grid, ions=ions, PP_list=pplist)
    hartree = Hartree()
    xc = XC(**xc_kwargs)
    ke = KEDF(**ke_kwargs)
    funcdicts = {'XC' :xc, 'HARTREE' :hartree, 'PSEUDO' :pseudo, 'KE' :ke}
    total_evaluator = TotalEvaluator(**funcdicts)
    return total_evaluator

In [6]:
def get_embed_evaluator(subcell):
    xc_kwargs = {'xc' : 'PBE'}
    ke_kwargs = {'kedf' : 'GGA', 'k_str' : 'revAPBEK'}
    xc = XC(**xc_kwargs)
    ke = KEDF(**ke_kwargs)
    #---------------------------------------------------------------
    emb_funcdicts = {'XC' :xc, 'KE' :ke}
    embed_evaluator = EmbedEvaluator(**emb_funcdicts)
    return embed_evaluator

In [7]:
def get_driver(prefix, ions, grid, pplist, index, cellsplit, graphtopo):
    mp = MP(comm = graphtopo.comm_sub, decomposition = graphtopo.decomposition)
    subcell = SubCell(ions, grid, index = index, cellsplit = cellsplit, mp = mp)
    # given a negative value which means will get from driver
    subcell.density[:] = -1.0
    embed_evaluator = get_embed_evaluator(subcell)
    cell_params = {'pseudopotentials' : pplist}
    params = {'system' : {'ecutwfc' : 600*ENERGY_CONV["eV"]["Hartree"]*2}}
    margs= {
            'evaluator' : embed_evaluator,
            'prefix' : prefix,
            'subcell' : subcell,
            'cell_params': cell_params,
            'params': params,
            'exttype' : 3, # 3 is XC embedded, 7 is without XC
            'mixer' : 0.7
            }
    engine = EngineQE()
    driver = DriverKS(engine = engine, **margs)
    return driver

In [8]:
# cellfile = 'h2o_2.xyz'
# subkeys = ['sub_ks_0', 'sub_ks_1']
# indices = [[0, 1, 2], [3, 4, 5]]
cellfile = 'h2o_1.xyz'
subkeys = ['sub_ks_0']
indices = [[0, 1, 2]]
opt = get_optimizer(cellfile, subkeys, indices)

********************************************************************************
Parallel version (MPI) on        1 processors
              eDFTpy Version : 0.0.post228+g280d5f0
               DFTpy Version : 1.0.post274+g17f16f3
********************************************************************************
GlobalCell grid [72 72 72]
setting key: H -> ./H_ONCV_PBE-1.2.upf
setting key: O -> ./O_ONCV_PBE-1.2.upf


In [9]:
opt.optimize()

Begin optimize
Optimization options :
{'econv': 3e-06,
 'maxiter': 80,
 'maxtime': 0,
 'ncheck': 2,
 'olevel': 2,
 'pconv': 3.0000000000000004e-08,
 'pconv_sub': array([3.e-08]),
 'sdft': 'sdft'}
Update density : 8.000000000000002
          Step    Energy(a.u.)            dE              dP        dC        Time(s)         
Norm of reidual density : 
[0.00864656]
Energy of reidual density : 
[0.79065876]
----------------------------------------------------------------------------------------------------
   Embed: 1       -6.829569089792E+00     -6.829569E+00   7.91E-01  8.65E-03  9.326077E-01    
----------------------------------------------------------------------------------------------------
Norm of reidual density : 
[0.01919708]
Energy of reidual density : 
[2.49549796]
----------------------------------------------------------------------------------------------------
   Embed: 2       -1.349536273918E+01     -6.665794E+00   2.50E+00  1.92E-02  1.708033E+00    
-----------------